In [1]:
#!/usr/bin/env python
# Jupyter Notebook 2: Retrieve & Insert Updated Clash Royale Data

import datetime
import json
import os
import pandas as pd
import requests
import urllib.parse
import hopsworks
import warnings
warnings.filterwarnings("ignore")

# ------------------------------------------------------------------------------
# 1. SET UP ENVIRONMENT AND HOPSWORKS LOGIN
# ------------------------------------------------------------------------------
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment below and read your key from a file or set it manually.
# os.environ["HOPSWORKS_API_KEY"] = 'YOUR-HOPSWORKS-API-KEY'

#Read your Hopsworks API Key
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key
project = hopsworks.login()
fs = project.get_feature_store()

2025-01-08 17:08:10,611 INFO: Initializing external client


2025-01-08 17:08:10,612 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-08 17:08:11,234 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700


In [2]:
import requests
import json
import os
import urllib.parse


def clean_battle_data(battle_data):
    # Define keys to retain
    desired_keys = {'name', 'level', 'elixirCost', 'rarity'}

    def filter_card_data(card):
        # Filter each card's data to retain only desired keys
        return {key: card[key] for key in card if key in desired_keys}

    def process_team_or_opponent(team_or_opponent):
        # Process each team or opponent's cards
        team_or_opponent['cards'] = [filter_card_data(card) for card in team_or_opponent['cards']]
        return team_or_opponent

    # Process the entire battle data
    for battle in battle_data:
        battle['team'] = [process_team_or_opponent(member) for member in battle['team']]
        battle['opponent'] = [process_team_or_opponent(member) for member in battle['opponent']]

    return battle_data

# Example usage with your JSON data
import json

# Assuming `battle_data` is your original JSON response loaded as a Python object
#cleaned_data = clean_battle_data(battle_data)

# Pretty-print the cleaned data
#print(json.dumps(cleaned_data, indent=2))

def get_player_data(player_tag, api_key):
    # URL-encode the player tag
    #encoded_tag = urllib.parse.quote(player_tag)

    # Define the API endpoint
    url = f'https://api.clashroyale.com/v1/players/{player_tag}/battlelog'

    # Set up the headers with the API key
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    # Make the GET request to the API
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error: {response.status_code}')
        return None

if __name__ == '__main__':
    # 1. Read the API key from the 'data/clash-royale-api-key.txt' file
    key_file_path = os.path.join('../data/clash-royale-api-key.txt')
    with open(key_file_path, 'r') as file:
        api_key = file.read().strip()

    # 2. Specify the player tag (URL-encoded # is often %23 in Clash Royale)
    player_tag = '%232LGY9G'  # Replace with the actual player tag, e.g. #2LGY9G -> %232LGY9G

    # 3. Call the API
    player_data = get_player_data(player_tag, api_key)
    if player_data:
        # 4. Clean the data
        cleaned_data = clean_battle_data(player_data)

        # 5. Save cleaned data to 'outputCR.json' in the 'data' folder
        output_path = os.path.join('../data/outputCR.json')
        with open(output_path, 'w', encoding='utf-16') as out_file:
            json.dump(cleaned_data, out_file, indent=2)

        # 6. Print the JSON file contents
        print(f"Saved cleaned data to: {output_path}\n")
        #print("Printing contents of outputCR.json:\n")
        with open(output_path, 'r', encoding='utf-16') as in_file:
            file_data = json.load(in_file)
            #print(json.dumps(file_data, indent=2))
    else:
        print("No data received from API. Please check your player tag or API key.")

Error: 403
No data received from API. Please check your player tag or API key.


In [3]:
import pandas as pd
import json
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Dictionary mapping card names to IDs
card_numbers = {
    "Archers": 1, "Archer Queen": 2, "Baby Dragon": 3, "Balloon": 4, "Bandit": 5, "Barbarians": 6,
    "Bats": 7, "Battle Healer": 8, "Battle Ram": 9, "Bomber": 10, "Bowler": 11, "Bush Goblins": 12,
    "Cannon Cart": 13, "Cursed Hog": 14, "Dark Prince": 15, "Dart Goblin": 16, "Electro Dragon": 17,
    "Electro Giant": 18, "Electro Spirit": 19, "Electro Wizard": 20, "Elite Barbarians": 21,
    "Elixir Blob": 22, "Elixir Golem": 23, "Elixir Golemite": 24, "Executioner": 25, "Firecracker": 26,
    "Fire Spirit": 27, "Fisherman": 28, "Flying Machine": 29, "Giant": 30, "Giant Skeleton": 31,
    "Goblin Brawler": 32, "Goblin Gang": 33, "Goblin Demolisher": 34, "Goblin Giant": 35,
    "Goblin Machine": 36, "Goblins": 37, "Goblinstein": 38, "Golden Knight": 39, "Golem": 40,
    "Golemite": 41, "Guardienne": 42, "Guards": 43, "Hog Rider": 44, "Hunter": 45, "Heal Spirit": 46,
    "Ice Golem": 47, "Ice Spirit": 48, "Ice Wizard": 49, "Inferno Dragon": 50, "Knight": 51,
    "Lava Hound": 52, "Lava Pup": 53, "Little Prince": 54, "Lumberjack": 55, "Magic Archer": 56,
    "Mega Knight": 57, "Mega Minion": 58, "Mighty Miner": 59, "Miner": 60, "Mini P.E.K.K.A.": 61,
    "Minion Horde": 62, "Minions": 63, "Monk": 64, "Mother Witch": 65, "Monster": 66, "Musketeer": 67,
    "Night Witch": 68, "P.E.K.K.A.": 69, "Phoenix": 70, "Reborn Phoenix": 71, "Prince": 72,
    "Princess": 73, "Ram Rider": 74, "Rascal Boy": 75, "Rascal Girl": 76, "Royal Ghost": 77,
    "Royal Giant": 78, "Royal Hogs": 79, "Royal Recruits": 80, "Skeleton Army": 81,
    "Skeleton Barrel": 82, "Skeleton Dragons": 83, "Skeleton King": 84, "Skeletons": 85, "Sparky": 86,
    "Spear Goblins": 87, "Suspicious Bush": 88, "Three Musketeers": 89, "Valkyrie": 90,
    "Wall Breakers": 91, "Witch": 92, "Wizard": 93, "Zappies": 94,"Bomb Tower": 95, "Cannon": 96, "Cannon Cart (broken)": 97, "Inferno Tower": 98, "Mortar": 99,
        "Tesla": 100, "X-Bow": 101,"Barbarian Hut": 102, "Elixir Collector": 103, "Furnace": 104, "Goblin Cage": 105,
        "Goblin Drill": 106, "Goblin Hut": 107, "Phoenix Egg": 108, "Tombstone": 109, "Arrows": 110, "Barbarian Barrel": 111, "Earthquake": 112, "Fireball": 113, "Freeze": 114,
        "Giant Snowball": 115, "Goblin Curse": 116, "Lightning": 117, "Poison": 118, "Rage": 119, "Rocket": 120, "Royal Delivery": 121, "The Log": 122, "Tornado": 123, "Void": 124, "Zap": 125,
        "Barbarian Barrel": 126, "Barbarian Hut": 127, "Battle Ram": 128, "Elixir Golem": 129,
        "Elixir Golemite": 130, "Furnace": 131, "Goblin Barrel": 132, "Goblin Cage": 133, "Goblin Curse": 134,
        "Goblin Drill": 135, "Goblin Giant": 136, "Goblin Hut": 137, "Golem": 138, "Graveyard": 139,
        "Lava Hound": 140, "Little Prince": 141, "Mother Witch": 142, "Night Witch": 143,
        "Phoenix Egg": 144, "Royal Delivery": 145, "Skeleton Barrel": 146, "Skeleton King": 147,
        "Suspicious Bush": 148, "Tombstone": 149, "Witch": 150,
        "Archers/Evolution": 155, "Barbarians/Evolution": 156, "Battle Ram/Evolution": 157,
        "Bats/Evolution": 158, "Bomber/Evolution": 159, "Cannon/Evolution": 160,
        "Electro Dragon/Evolution": 161, "Firecracker/Evolution": 162, "Giant Snowball/Evolution": 163,
        "Goblin Barrel/Evolution": 164, "Goblin Cage/Evolution": 165, "Goblin Drill/Evolution": 166,
        "Goblin Giant/Evolution": 167, "Ice Spirit/Evolution": 168, "Knight/Evolution": 169,
        "Mega Knight/Evolution": 170, "Mortar/Evolution": 171, "Musketeer/Evolution": 172,
        "P.E.K.K.A/Evolution": 173, "Royal Giant/Evolution": 174, "Royal Recruits/Evolution": 175,
        "Skeletons/Evolution": 176, "Tesla/Evolution": 177, "Valkyrie/Evolution": 178,
        "Wall Breakers/Evolution": 179, "Wizard/Evolution": 180, "Zap/Evolution": 181

}


# One-hot encoder setup
num_choices = 181  # Total possible card IDs # 116 actual cards
encoder = OneHotEncoder(categories=[range(num_choices)], sparse_output=False)

# Assuming the provided JSON is saved in 'outputCR.json'
try:
    with open('../data/outputCR.json', 'r', encoding='utf-16') as f:
        battle_data = json.load(f)  # Load the JSON data
except FileNotFoundError:
    print("File not found. Ensure 'outputCR.json' exists in the directory.")
    battle_data = None
except json.JSONDecodeError as e:
    print(f"Invalid JSON format: {e}")
    battle_data = None

if battle_data:
    # Function to process a single player's data
    def process_player_data(player, is_team=True):
        # Convert card names to IDs and handle missing cases
        deck = [card_numbers.get(card['name'], -1) for card in player['cards']]
        # Filter out invalid card IDs (-1)
        deck = [card for card in deck if card >= 0]

        # One-hot encode the deck
        if len(deck) > 0:
            deck_one_hot = encoder.fit_transform(np.array(deck).reshape(-1, 1)).sum(axis=0)
        else:
            deck_one_hot = np.zeros(num_choices)

        trophy_change = player.get('trophyChange', 0) * (1 if is_team else -1)
        return {
            'Player Name': player['name'],
            'Deck': list(deck_one_hot), # (One-Hot Encoded)
            'Result': '',
        }

    # Collect all matches
    all_matches = []

    for match in battle_data:
        team = match['team'][0]
        opponent = match['opponent'][0]
        team_data = process_player_data(team, is_team=True)
        opponent_data = process_player_data(opponent, is_team=False)

        # Add Win or Lose based on crowns
        if team['crowns'] > opponent['crowns']:
            team_data['Result'] = 1 #'WIN'
            opponent_data['Result'] = 0 #'LOSE'
        elif team['crowns'] < opponent['crowns']:
            team_data['Result'] = 0 #'LOSE'
            opponent_data['Result'] = 1 #'WIN'
        else:
            team_data['Result'] = 2#'DRAW'
            opponent_data['Result'] =2 #'DRAW'

        all_matches.append(team_data)
        all_matches.append(opponent_data)

    # Create the DataFrame
    df = pd.DataFrame(all_matches)

    # Display the DataFrame
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    print(df)
else:
    print("No battle data available.")

        Player Name  \
0           Diwel26   
1      ARGENTINO_09   
2           Diwel26   
3           Shmuck5   
4           Diwel26   
5     b a l o o o n   
6           Diwel26   
7             diego   
8           Diwel26   
9            槍在手跟我走   
10          Diwel26   
11             More   
12          Diwel26   
13      حسین شهبازی   
14          Diwel26   
15      killergamer   
16          Diwel26   
17            J.MR✓   
18          Diwel26   
19            원주민준우   
20          Diwel26   
21              갓대현   
22          Diwel26   
23    papillon d'or   
24          Diwel26   
25            felix   
26          Diwel26   
27         Пельмень   
28          Diwel26   
29             toby   
30          Diwel26   
31             Bach   
32          Diwel26   
33  sirop de fraise   
34          Diwel26   
35          Edu638™   
36          Diwel26   
37           vaccoc   
38          Diwel26   
39              Gui   
40          Diwel26   
41           Arthur   
42         

In [4]:
# Process the data to create features and target
if battle_data:
    # Function to process a single player's data
    def process_player_data(player):
        # Convert card names to IDs and handle missing cases
        deck = [card_numbers.get(card['name'], -1) for card in player['cards']]
        # Filter out invalid card IDs (-1)
        deck = [card for card in deck if card >= 0]

        # One-hot encode the deck
        if len(deck) > 0:
            deck_one_hot = encoder.fit_transform(np.array(deck).reshape(-1, 1)).sum(axis=0)
        else:
            deck_one_hot = np.zeros(num_choices)

        return list(deck_one_hot)  # Return the one-hot encoded deck as a list

    # Collect all matches
    all_matches = []

    for match in battle_data:
        # Extract team and opponent data
        team = match['team'][0]
        opponent = match['opponent'][0]

        # Process the decks
        deck1 = process_player_data(team)
        deck2 = process_player_data(opponent)

        # Determine the result (1 if deck1 wins, 0 if deck2 wins)
        if team['crowns'] > opponent['crowns']:
            result = 1  # deck1 wins
        elif team['crowns'] < opponent['crowns']:
            result = 0  # deck2 wins
        else:
            continue  # Skip matches that are draws

        # Add the match data as a single row
        all_matches.append({
            'Player1': team['name'],
            'Player2': opponent['name'],
            'Deck1': deck1,
            'Deck2': deck2,
            'Result': result  # 1 if Deck1 wins, 0 if Deck2 wins
        })

    # Create the DataFrame
    df = pd.DataFrame(all_matches)

df

,Player1,Player2,Deck1,Deck2,Result
0,Diwel26,ARGENTINO_09,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",1
1,Diwel26,Shmuck5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...]",0
2,Diwel26,b a l o o o n,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0
3,Diwel26,diego,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [5]:
# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

# 3. Log in to your Hopsworks project
project = hopsworks.login()  # or hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

# 4. Suppose we already have a DataFrame named df (e.g., from the previous step)
#    that has columns like: "player_name", "deck", "crowns", "result", "battle_time", etc.
#    Make sure df is defined before this script or above in the same script:
# df = ...

# 5. Create (or get) a feature group
cr_feature_group = fs.get_or_create_feature_group(
    name="clash_royale_onehotencoding",
    version=5,
    description="Clash Royale cleaned match features",
    primary_key=["Player1"],  # columns that uniquely identify each row
    online_enabled=True
)

# 6. Insert data into the feature group
cr_feature_group.insert(df)

print("DataFrame inserted into Hopsworks Feature Store successfully!")

2025-01-08 17:08:12,684 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-01-08 17:08:12,687 INFO: Initializing external client


2025-01-08 17:08:12,687 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-08 17:08:13,244 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700


Uploading Dataframe: 0.00% |          | Rows 0/30 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 6.67% |▋         | Rows 2/30 | Elapsed Time: 00:01 | Remaining Time: 00:14

Uploading Dataframe: 100.00% |██████████| Rows 30/30 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: clash_royale_onehotencoding_5_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1175700/jobs/named/clash_royale_onehotencoding_5_offline_fg_materialization/executions
DataFrame inserted into Hopsworks Feature Store successfully!
